# Homework 4 - More trains (Part I & II)

__Hand-in:__

- __Due: 12.05.2020 23:59:59 CET__
- `git push` your final verion to your group's Renku repository before the due
- check if `Dockerfile`, `environment.yml` and `requirements.txt` are properly written
- add necessary comments and discussion to make your codes readable

For this homework, you will be working with the real-time streams of the NS, the train company of the Netherlands. You can see an example webpage that uses the same streams to display the train information on a map: https://spoorkaart.mwnn.nl/ . 

To help you and avoid having too many connections to the NS streaming servers, we have setup a service that collects the streams and pushes them to our Kafka instance. The related topics are: 

`ndovloketnl-arrivals`: For each arrival of a train in a station, describe the previous and next station, time of arrival (planned and actual), track number,...

`ndovloketnl-departures`: For each departure of a train from a station, describe the previous and next station, time of departure (planned and actual), track number,...

`ndovloketnl-gps`: For each train, describe the current location, speed, bearing.

The events are serialized in JSON (actually converted from XML), with properties in their original language. Google translate could help you understand all of them, but we will provide you with some useful mappings.

---
## Create a Kafka client

In [1]:
import os
from pykafka import KafkaClient
from pykafka.common import OffsetType
import json

username = os.environ['JUPYTERHUB_USER']

ZOOKEEPER_QUORUM = 'iccluster044.iccluster.epfl.ch:2181,'\
                   'iccluster054.iccluster.epfl.ch:2181,'\
                   'iccluster059.iccluster.epfl.ch:2181'

client = KafkaClient(zookeeper_hosts=ZOOKEEPER_QUORUM)

---

## Part I - Live Plot (20 points / 50)

The goal of this part is to obtain an interactive plot use the train positions from the GPS stream. We encourage you to use the examples from last week to achieve the expected result.

First, let's write a function to decode the messages from the `ndovloketnl-gps` topic.

In [2]:
import json
from pykafka.common import OffsetType

example_gps = client.topics[b'ndovloketnl-gps'].get_simple_consumer(
    auto_offset_reset=OffsetType.EARLIEST,
    reset_offset_on_start=True
).consume()
json.loads(example_gps.value)

{'tns3:ArrayOfTreinLocation': {'@xmlns:tns3': 'http://schemas.datacontract.org/2004/07/Cognos.Infrastructure.Models',
  'tns3:TreinLocation': [{'tns3:TreinNummer': '15867',
    'tns3:TreinMaterieelDelen': [{'tns3:MaterieelDeelNummer': '2451',
      'tns3:Materieelvolgnummer': '1',
      'tns3:GpsDatumTijd': '2020-04-23T17:06:31Z',
      'tns3:Orientatie': '0',
      'tns3:Bron': 'NTT',
      'tns3:Fix': '1',
      'tns3:Berichttype': None,
      'tns3:Longitude': '5.1597604696057',
      'tns3:Latitude': '52.273841159003',
      'tns3:Elevation': '0.0',
      'tns3:Snelheid': '90.0',
      'tns3:Richting': '166.6',
      'tns3:Hdop': '2.38',
      'tns3:AantalSatelieten': '0'},
     {'tns3:MaterieelDeelNummer': '2430',
      'tns3:Materieelvolgnummer': '2',
      'tns3:GpsDatumTijd': '2020-04-23T17:06:34Z',
      'tns3:Orientatie': '0',
      'tns3:Bron': 'NTT',
      'tns3:Fix': '1',
      'tns3:Berichttype': None,
      'tns3:Longitude': '5.1597830062141',
      'tns3:Latitude': '52.

We can see that the message has the following structure:

```
{
  'tns3:ArrayOfTreinLocation': {
    'tns3:TreinLocation': [
      <train_info_1>,
      <train_info_2>,
      ...
    ]
  }
}
```

With the `<train_info_x>` messages containing:
- `tns3:TreinNummer`: the train number. This number is used in passenger information displays.
- `tns3:MaterieelDeelNummer`: the train car number. It identifies the physical train car.
- `tns3:Materieelvolgnummer`: the car position. 1 is the car in front of the train, 2 the next one, etc.
- `tns3:GpsDatumTijd`: the datetime given by the GPS.
- `tns3:Latitude`, `tns3:Longitude`, `tns3:Elevation`: 3D coordinates given by the GPS.
- `tns3:Snelheid`: speed, most likely given by the GPS.
- `tns3:Richting`: heading, most likely given by the GPS.
- `tns3:AantalSatelieten`: number of GPS satellites in view.

We also notice that when a train is composed of multiple cars, the position is given in an array, with the position of all individual cars.

**Question I.a. (5/20)** Write a function `extract_gps_data` which takes the message as input and extracts the train number, train car and GPS data from the source messages. Using this function, you should be able to obtain the example table, or something similar:

| timestamp | train_number | car_number | car_position | longitude | latitude | elevation | heading | speed |
|:---------:|:------------:|:----------:|:------------:|:---------:|:--------:|:---------:|:-------:|:-----:|
|    ...    |      ...     |     ...    |      ...     |    ...    |    ...   |    ...    |   ...   |  ...  |

__Hints:__
- The messages can be occaionally are empty, for example, `tns3:ArrayOfTreinLocation` or `tns3:TreinLocation` can be empty.
- Not every message shares exactly the same structure, for example, `tns3:TreinMaterieelDelen` may be a list but not always
- You may find Python disctionary [get(key, default)](https://docs.python.org/3.7/library/stdtypes.html#dict.get) method helpful.

**Methodology**: Messages are a form of nested dictionaries and lists, which we need to loop through. There are some empty messages, this needs to be checked first.

In [3]:
import numpy as np
import pandas as pd

def extract_gps_data(msg):

    #Preallocate
    gps_data = []
    
    #Check if 'tns3:ArrayOfTreinLocation'.'tns3:TreinLocation' is an empty message.
    if not msg.get('tns3:ArrayOfTreinLocation').get('tns3:TreinLocation'):
        print('Message is empty')
    else:
        trains = msg.get('tns3:ArrayOfTreinLocation').get('tns3:TreinLocation')
        for train in trains:
            cars = train.get('tns3:TreinMaterieelDelen')
            #Some train cars are not list, so conversion is needed.
            if(type(cars) != list):
                cars = [cars]
            #Create a table by appending relevant parameters.   
            for car in cars:
                gps_data.append([                    
                car['tns3:GpsDatumTijd'],
                train['tns3:TreinNummer'],
                car['tns3:MaterieelDeelNummer'],
                car['tns3:Materieelvolgnummer'],
                car['tns3:Longitude'],
                car['tns3:Latitude'],
                car['tns3:Elevation'],
                car['tns3:Richting'],
                car['tns3:Snelheid'],])
    return gps_data

In [4]:
# Example results from "extract_gps_data"
import numpy as np
import pandas as pd

data = pd.DataFrame(
    data=extract_gps_data(json.loads(example_gps.value)),
    columns=['timestamp', 'train_number', 'car_number', 'car_position', 
             'longitude', 'latitude', 'elevation', 'heading', 'speed']
)
data.head(20)

,timestamp,train_number,car_number,car_position,longitude,latitude,elevation,heading,speed
0,2020-04-23T17:06:31Z,15867,2451,1,5.1597604696057,52.273841159003,0.0,166.6,90.0
1,2020-04-23T17:06:34Z,15867,2430,2,5.1597830062141,52.273789073293,0.0,166.6,86.4
2,2020-04-23T17:06:36Z,14667,2728,1,5.322429,52.41208,0.62,61.88,133.789
3,2020-04-23T17:06:32Z,14667,2332,2,5.319623,52.411064,0.62,56.88,129.182
4,2020-04-23T17:06:36Z,14667,2740,3,5.320444,52.411377,0.58,58.44,133.808
5,2020-04-23T17:06:31Z,6662,2513,1,5.79994583333,51.8246003333,0.0,79.17,66.0
6,2020-04-23T17:06:35.8Z,6662,2507,2,5.799830330535919,51.82456617400802,14.0,79.11,76.0
7,2020-04-23T17:06:34Z,6966,2618,1,5.2605806017145,51.824128392772,0.0,353.6,104.4
8,2020-04-23T17:06:31Z,8161,8723,1,6.55242116667,52.9407786667,0.0,12.3,125.0
9,2020-04-23T17:06:31Z,5668,2710,1,5.892624,52.408997,0.63,232.0,11.646


**Question I.b (15/20)** Make a live plot of the train positions.

You can do so by using `bokeh`; use last week's lab as an example.

See also: https://docs.bokeh.org/en/latest/docs/user_guide/geo.html#tile-provider-maps

You can compare your plot to one of the live services: https://spoorkaart.mwnn.nl/, http://treinenradar.nl/

__Q (1/15)__ To plot points with GPS location information on bokeh's map, we need a transoformer. What is the following transformer capable of? Check `bokeh`'s documenation on [Tile Provider Maps](https://docs.bokeh.org/en/latest/docs/user_guide/geo.html#tile-provider-maps).

In [5]:
from pyproj import Transformer
transformer = Transformer.from_proj("EPSG:4326", "EPSG:3857", always_xy=True)

__Answer:__ The transformer takes as input coordinates in the geographical coordinate system (GCS, also called geodetic coordinate system; EPSG:4326) and transforms them into a bokeh friendly projected coordinate system, which is used for map rendering (EPSG:3857). If always_xy is True, then the transformer keeps the traditional GIS order (longitude, latitude).

To perform a transfromation, you need to use the method `Transfromer.transform`, please check [here](https://pyproj4.github.io/pyproj/stable/api/transformer.html?highlight=transformer#pyproj.transformer.Transformer.transform).

__Q (14/15)__ Let's make the plot.

**Care should be taken for the following point:**
- We expect the train positions to fall on rail tracks on the map. Showing each train as a circle is good enough. Check [Scatter Markers](https://docs.bokeh.org/en/2.0.2/docs/user_guide/plotting.html?highlight=scatter#scatter-markers).
- One train may have many cars. You do not need to show every car on the map, please keep only car whose `car_position` equals to '1'.
- Provide an interactive label with the train number (we do not expect train type, as this needs to be recovered from other sources). Check [Hovertool](https://docs.bokeh.org/en/2.0.1/docs/user_guide/tools.html#hovertool).

**You can get bonus points if you make followings happen on your plot:**
- Trains on the map should not appear/disappear when data is absent for a few messages.
- Find a way to show where the train is heading.
- Add any other pieces of information that may be of interest to users.

In [24]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
from bokeh.tile_providers import get_provider, OSM
from bokeh.models.annotations import Title

import time

output_notebook()

Loading BokehJS ...

Create a simple consumer for `ndovloketnl-gps`, which consumes the latest information from the stream.

In [30]:
consumer = client.topics[b'ndovloketnl-gps'].get_simple_consumer(
    auto_offset_reset=OffsetType.LATEST,
    reset_offset_on_start=True, 
    auto_commit_enable=True,
    auto_commit_interval_ms=2000
)

In [31]:
gps_data = pd.DataFrame(data=extract_gps_data(json.loads(message.value)),columns=['timestamp', 'train_number', 'car_number', 'car_position','longitude', 'latitude', 'elevation', 'heading', 'speed'])       
#Filtering by car_position equal to one
gps_data = gps_data[gps_data['car_position'] == '1']
#Extract longitutde coordinates
x = gps_data['longitude'].tolist()
xx = [float(l) for l in x]
#Extract latitude coordinates
y = gps_data['latitude'].tolist()
yy = [float(l) for l in y]
xxx, yyy = transformer.transform(xx, yy)
#Extract train number
tn = gps_data['train_number'].tolist()
#Extract speed of train in km/h
sp = gps_data['speed'].tolist()

In [33]:
xx = []
yy = []
tn = []

#Define the data source for live plot
source = ColumnDataSource(data=dict(
    x=xx,
    y=yy,
    tn=tn,
    sp=sp))

#Set up the Hoverplot, so that it shows the train number and speed of each train.
TOOLTIPS = [
    ("Train number", "@tn"),
    ("Speed (in km/h)","@sp")
]

#Create the live plot
tile_provider = get_provider(OSM)
p = figure(x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS)
p.add_tile(tile_provider)

p.circle('x', 'y', source=source, size=5, line_color="navy", fill_color="orange", alpha=0.8)

#Make the plot centered around Amsterdam
x_min, y_min = transformer.transform(4.4, 52.2)
x_max, y_max = transformer.transform(5.4, 52.6)
p.x_range = Range1d(x_min, x_max)
p.y_range = Range1d(y_min, y_max)
title = "Live plot of trains in Amsterdam area"
p.title = Title(text=title)

handle = show(p, notebook_handle=True)

In [34]:
try:
    for message in consumer:
        if message is not None:
            gps_data = pd.DataFrame(
            data=extract_gps_data(json.loads(message.value)),columns=['timestamp', 'train_number', 'car_number', 'car_position','longitude', 'latitude', 'elevation', 'heading', 'speed'])       
            gps_data = gps_data[gps_data['car_position'] == '1']  # Filtering by car_position equal to one
            x = gps_data['longitude'].tolist()
            xx = [float(l) for l in x]
            y = gps_data['latitude'].tolist()
            yy = [float(l) for l in y]
            xxx, yyy = transformer.transform(xx, yy)
            tn = gps_data['train_number'].tolist()
            sp = gps_data['speed'].tolist()
            source.stream({'x':xxx, 'y': yyy, 'tn':tn, 'sp': sp}, rollover=20)
            p.circle('x', 'y', source=source, size=5, line_color="navy", fill_color="orange", alpha=0.8)
            push_notebook(handle=handle)
            time.sleep(0.1)
except KeyboardInterrupt:
    print('stopped')
    pass

stopped


**Discussion**: We have successfully implemented a live plot, which shows the current trains in the Amsterdam area. The plot only visualizes the first car of each train (as circle), and if the user hovers over a circle, the **train number** and **speed in km/h** will be shown.

---

# Part II - Locate Message (10 points / 50)

After you finish this part, you are able to locate the message given a specific timestamp.

You can find below a helper function to read a message at a specific offset from a Kafka topic.

In [35]:
def fetch_message_at(topic, offset):
    if isinstance(topic, str):
        topic = topic.encode('utf-8')
    t = client.topics[topic]
    consumer = t.get_simple_consumer()
    p = list(consumer.partitions.values())[0]
    consumer.reset_offsets([(p,int(offset)-1)], )
    return consumer.consume()

In [36]:
msg = fetch_message_at(b'ndovloketnl-gps', 31243)

In [37]:
msg.offset

31243

In [15]:
msg.value

b'{"tns3:ArrayOfTreinLocation": {"@xmlns:tns3": "http://schemas.datacontract.org/2004/07/Cognos.Infrastructure.Models", "tns3:TreinLocation": [{"tns3:TreinNummer": "1685", "tns3:TreinMaterieelDelen": [{"tns3:MaterieelDeelNummer": "4058", "tns3:Materieelvolgnummer": "0", "tns3:GpsDatumTijd": "2020-04-26T04:36:23Z", "tns3:Orientatie": "0", "tns3:Bron": "NTT", "tns3:Fix": "1", "tns3:Berichttype": null, "tns3:Longitude": "5.3565625", "tns3:Latitude": "52.154243", "tns3:Elevation": "0.0", "tns3:Snelheid": "0", "tns3:Richting": "0.0", "tns3:Hdop": "4.3", "tns3:AantalSatelieten": "9"}, {"tns3:MaterieelDeelNummer": "4204", "tns3:Materieelvolgnummer": "0", "tns3:GpsDatumTijd": "2020-04-26T04:36:29Z", "tns3:Orientatie": "0", "tns3:Bron": "NTT", "tns3:Fix": "1", "tns3:Berichttype": null, "tns3:Longitude": "5.35796566667", "tns3:Latitude": "52.1534048333", "tns3:Elevation": "0.0", "tns3:Snelheid": "0", "tns3:Richting": "0.0", "tns3:Hdop": "4.3", "tns3:AantalSatelieten": "10"}]}, {"tns3:TreinNummer

**Question II.a (5/10)** Write a function to extract the median timestamp from a message of the `ndovloketnl-gps` topic. You can reuse the `extract_gps_data` function from part I.

In [38]:
topic = client.topics[b'ndovloketnl-gps']
example_gps = topic.get_simple_consumer(
    auto_offset_reset=OffsetType.EARLIEST,
    reset_offset_on_start=True
).consume()

In [39]:
earliest = topic.earliest_available_offsets()
earliest[0].offset[0]
latest = topic.latest_available_offsets()
latest[0].offset[0]

164557

In [56]:
# Answer II.a
import pandas as pd
import numpy as np

def extract_gps_time_approx(msg):
    #Use extract_gps_data to create df from which the timestamp can be extracted.
    data = pd.DataFrame(data=extract_gps_data(msg),columns=['timestamp', 'train_number', 
                                           'car_number', 'car_position','longitude', 'latitude', 'elevation', 'heading', 'speed'])
    #Convert timestamp to datetime64 and then to numeric object
    time_datetime_format = np.array(pd.to_datetime(data['timestamp']), dtype=np.datetime64)
    time_numeric_format = pd.to_numeric(time_datetime_format)
    #Compute median and reconvert to datetime64 object
    median_numeric_format = np.median(time_numeric_format)
    median_datetime_format = pd.to_datetime(median_numeric_format) 
    median_ts = np.datetime64(median_datetime_format)
    #Output Timestamp object
    return pd.Timestamp(median_ts)

In [57]:
extract_gps_time_approx(json.loads(example_gps.value))

Timestamp('2020-04-23 17:06:34')

In [20]:
# Example results from `extract_gps_time_approx`
extract_gps_time_approx(json.loads(example_gps.value))

Timestamp('2020-04-23 17:06:34')

**Question II.b (5/10)** Using `fetch_message_at` and `extract_gps_time_approx`, write a function named `search_gps` to find the "first" offset for a given timestamp in the `ndovloketnl-gps` topic. You function should use [Binary Search Algorithm](https://en.wikipedia.org/wiki/Binary_search_algorithm).

More preciseley, if we note `offset = search_gps(ts)` where `ts` is a timestamp, then we have:
```
ts <= extract_gps_time_approx(fetch_message_at('ndovloketnl-gps', offset))

extract_gps_time_approx(fetch_message_at('ndovloketnl-gps', offset - 1)) < ts
```

In [21]:
from datetime import datetime

def search_gps(findTimeStr):
    #Convert timestamp to datetime64 object, extract lower and upper offsets.
    findTime = findTimeStr.to_datetime64()
    lower = topic.earliest_available_offsets()[0].offset[0]
    upper = topic.latest_available_offsets()[0].offset[0]
    result = -1
    #Apply binary search algorithm
    while(lower <= upper):
        mid = lower + (upper - lower) // 2
        msg = fetch_message_at(b'ndovloketnl-gps', mid)
        t = extract_gps_time_approx(json.loads(msg.value))
        d = pd.to_datetime(t)
        d = d.replace(hour = 0, minute=0, second=0, microsecond=0)
        ts = pd.Timestamp(d)
        if findTimeStr == ts:
            result = mid
            upper= mid - 1
        elif findTimeStr < ts:
            upper = mid - 1
        else:
            lower = mid + 1
    
    return result   

In [22]:
# Example results from `search_gps`
offset = search_gps(pd.Timestamp('2020-04-30'))
offset

58197

In [23]:
# Verify that offset returned above returns a timestamp on or after 2020-04-23 (replace <--OFFSET-->)
extract_gps_time_approx(json.loads(fetch_message_at('ndovloketnl-gps', offset).value))

Timestamp('2020-04-30 00:00:04.500000')

In [24]:
# Verify that offset returned above returns a timestamp before 2020-04-23 (replace <--OFFSET-->)
extract_gps_time_approx(json.loads(fetch_message_at('ndovloketnl-gps', offset - 1).value))

Timestamp('2020-04-29 23:59:55.500000')

**Discussion**: As we have verified with the example `Timestamp('2020-04-30')`, our `search_gps()` function returns the first offset of a timestamp.